In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `Tech` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate whe

In [3]:
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install bitsandbytes accelerate xformers einops

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = transformers.BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.0 MB/s eta 0:00:00


In [5]:
%%time
import pandas as pd
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re


def extract_list(llm_output):
    llm_output = llm_output.replace("\n", " ")
    items = re.findall(r"\d+\.\s*([\w\s&-]+)", llm_output)
    return [item.strip() for item in items if item]


def generate_category(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_length=550)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

financial_needs_raw = generate_category("List 50 common financial needs for businesses and individuals like Home Loan, Retirement Saving, ETFs:")
preferences_raw = generate_category("Provide 50 different customer preferences for spending:")
occupations_raw = generate_category("Generate 50 realistic occupations in finance, retail, healthcare, and tech:")
social_intents_raw = generate_category("List 50 social media financial sentiments like Loving the new fashion trends this season! etc:")
educations_raw = generate_category("Generate 50 realistic education levels like masters btech student:")


financial_needs = extract_list(financial_needs_raw)
preferences = extract_list(preferences_raw)
occupations = extract_list(occupations_raw)
social_intents = extract_list(social_intents_raw)
educations = extract_list(educations_raw)


print("Financial Needs:", financial_needs)
print("Preferences:", preferences)
print("Occupations:", occupations)
print("Social Intents:", social_intents)
print("Educations:", educations)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Financial Needs: ['Home Loan 2', 'ETFs 4', 'Budgeting 6', 'Credit Score 8', 'Insurance 10', 'Credit Card 12', 'Stocks 14', 'Mutual Funds 16', 'Personal Loan 18', 'Business Credit 20', 'Business Tax Planning 22', 'Business Credit Card 24', 'Business Stocks 26', 'Business Mutual Funds 28', 'Business Retirement Planning 30', 'Business Estate Planning 32', 'Business Debt Management 34', 'Business Credit Management 36', 'Business Insurance Management 38', 'Business Credit Card Management 40', 'Business Stocks Management 42', 'Business Mutual Funds Management 44', 'Business Retirement Management 46', 'Business Estate Management 48', 'Business Debt Management 50']
Preferences: ['Quality over quantity 2', 'Convenient location 4', 'Excellent customer service 6', 'Fast and reliable delivery 8', 'Personalized recommendations 10', 'Discounts and promotions 12', 'Sustainable and eco-friendly products 14', 'Gluten-free and vegan options 16', 'High-tech and innovative products 18', 'Positive reviews 

In [8]:
import re

def clean_list(items):
    return [re.sub(r'\s*\d+$', '', item).strip() for item in items]

financial_needs = clean_list(financial_needs)
preferences = clean_list(preferences)
occupations = clean_list(occupations)
social_intents = clean_list(social_intents)
educations = clean_list(educations)
payment_modes = ["Credit Card", "Wire Transfer", "PayPal", "Cash", "Cryptocurrency"]
print(len(financial_needs))
print(len(preferences))
print(len(occupations))
print(len(social_intents))
print(len(payment_modes))
print(len(educations))

print("Financial Needs:", financial_needs)
print("Preferences:", preferences)
print("Occupations:", occupations)
print("Social Intents:", social_intents)
print("Payment Modes:", payment_modes)
print("Educations:", educations)

25
26
38
36
5
25
Financial Needs: ['Home Loan', 'ETFs', 'Budgeting', 'Credit Score', 'Insurance', 'Credit Card', 'Stocks', 'Mutual Funds', 'Personal Loan', 'Business Credit', 'Business Tax Planning', 'Business Credit Card', 'Business Stocks', 'Business Mutual Funds', 'Business Retirement Planning', 'Business Estate Planning', 'Business Debt Management', 'Business Credit Management', 'Business Insurance Management', 'Business Credit Card Management', 'Business Stocks Management', 'Business Mutual Funds Management', 'Business Retirement Management', 'Business Estate Management', 'Business Debt Management']
Preferences: ['Quality over quantity', 'Convenient location', 'Excellent customer service', 'Fast and reliable delivery', 'Personalized recommendations', 'Discounts and promotions', 'Sustainable and eco-friendly products', 'Gluten-free and vegan options', 'High-tech and innovative products', 'Positive reviews and ratings', 'In-store pickup', 'Payment options', 'Free shipping', 'Product

In [15]:
%%time
import pandas as pd
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

num_samples = 100000

def generate_random_data(num_samples=500):
    customer_profiles = []

    for i in range(num_samples):
        cust_id = f"CUST2025{i}"
        org_id = f"ORG_US_{random.randint(1, 1000)}"

        customer_profiles.append([
            cust_id, random.randint(22, 65), random.choice(["M", "F", "Non-binary"]),
            random.choice(["New York", "San Francisco", "Los Angeles", "Chicago", "Miami"]),
            random.choice(financial_needs), random.choice(social_intents),
            random.choice(preferences), random.randint(40000, 300000),
            random.choice(occupations)
        ])

    return customer_profiles


profiles = generate_random_data(num_samples=num_samples)

df_customers = pd.DataFrame(profiles, columns=["Customer_Id", "Age", "Gender", "Location", "Financial_Need", "Social_Intent", "Interests", "Income", "Occupation"])

df_customers.to_csv("customer_profiles.csv", index=False)


print("Dataset generated & saved successfully!")


Dataset generated & saved successfully!
CPU times: user 2.32 s, sys: 43.5 ms, total: 2.37 s
Wall time: 2.42 s


In [16]:
customer_data = pd.read_csv('/content/customer_profiles.csv').head(100)
print(customer_data.shape)
customer_data.head()

(100, 9)


,Customer_Id,Age,Gender,Location,Financial_Need,Social_Intent,Interests,Income,Occupation
0,CUST20250,49,F,Chicago,Mutual Funds,Can,Product recommendations based on time of day,55860,Financial Analyst
1,CUST20251,29,Non-binary,Los Angeles,Stocks,Pumped up for the upcoming dance party,Product recommendations based on purchase history,207897,Risk Manager
2,CUST20252,35,M,Chicago,Business Debt Management,Excited to try out the latest beauty products,Product recommendations based on time of day,44904,Insurance Underwriter
3,CUST20253,24,M,Los Angeles,Business Tax Planning,Can,Product recommendations based on search history,42476,Sales Associate
4,CUST20254,55,F,New York,ETFs,Can,Convenient location,219634,Financial Planner


In [17]:
financial_products = [
    # Credit Cards
    {"product": "Premium Credit Card", "credit_line": "$50,000", "tenure": "5 years", "interest_rate": "14%", "offers": "Luxury Shopping, Travel Rewards, Priority Pass"},
    {"product": "Business Credit Card", "credit_line": "$75,000", "tenure": "5 years", "interest_rate": "12%", "offers": "Higher Credit Limit, Airport Lounge Access, Free Insurance"},
    {"product": "Student Credit Card", "credit_line": "$5,000", "tenure": "1 year", "interest_rate": "24%", "offers": "Low Annual Fee, Cashback on Education & Books"},
    {"product": "Retail Credit Card", "credit_line": "$15,000", "tenure": "3 years", "interest_rate": "20%", "offers": "Shopping Rewards, EMI Options"},
    {"product": "Travel Credit Card", "credit_line": "$30,000", "tenure": "4 years", "interest_rate": "16%", "offers": "Flight Miles, Hotel Discounts, Lounge Access"},
  ]


In [28]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        output = model(**inputs).logits.cpu().numpy().flatten()
    return output


def recommend_product_vector(customer_row):
    #customer_text = f"Financial Need: {customer_row['Financial_Need']}, Interests: {customer_row['Interests']}, Income: {customer_row['Income']}, Occupation: {customer_row['Occupation']}"
    customer_text = f"{customer_row['Financial_Need']}, {customer_row['Interests']}, {customer_row['Income']}, {customer_row['Occupation']}, {customer_row['Age']},  {customer_row['Social_Intent']} "
    customer_embedding = get_embedding(customer_text)

    product_embeddings = []
    max_dim = 0

    for product in financial_products:
        product_text = f"{product['product']} {product['offers']}"
        embedding = get_embedding(product_text)
        product_embeddings.append(embedding)
        max_dim = max(max_dim, embedding.shape[0])


    def pad_or_truncate(arr, size):
        if arr.shape[0] > size:
            return arr[:size]
        else:
            return np.pad(arr, (0, size - arr.shape[0]))

    customer_embedding = pad_or_truncate(customer_embedding, max_dim)
    product_embeddings = np.array([pad_or_truncate(e, max_dim) for e in product_embeddings])  # Convert to fixed-size 2D array


    similarities = cosine_similarity(customer_embedding.reshape(1, -1), product_embeddings)
    best_match_idx = similarities.argmax()


    return financial_products[best_match_idx]


In [29]:
%%time
for j in range(5):
  custrow = customer_data.iloc[j, :]
  print(custrow.T)
  print(recommend_product_vector(custrow))
  print()

Customer_Id                                          CUST20250
Age                                                         49
Gender                                                       F
Location                                               Chicago
Financial_Need                                    Mutual Funds
Social_Intent                                              Can
Interests         Product recommendations based on time of day
Income                                                   55860
Occupation                                   Financial Analyst
Name: 0, dtype: object
{'product': 'Premium Credit Card', 'credit_line': '$50,000', 'tenure': '5 years', 'interest_rate': '14%', 'offers': 'Luxury Shopping, Travel Rewards, Priority Pass'}

Customer_Id                                               CUST20251
Age                                                              29
Gender                                                   Non-binary
Location                               